1. Modify the multipath model from the previous homework assignments so that the transmitter is an array of Nt elements and the receiver is an array of Nr elements. Generate a sequence of a few hundred channel matrices for 4 transmit and 4 receive antennas with optimal element spacing (0.4λ) as the transmitter or receiver moves. Compute the capacity for each channel realization based on the water-filling solution for a given “SNR” (PT/σ2 w) and average over the channels.

a. Plot the average channel capacity as a function of PT/σ2 w over an appropriate range such as 0 to 30 dB. Use a semilogy scale for the vertical axis (capacity) and dB for the horizontal axis (SNR). Include on the same plot the SISO capacity for one transmit and one receive element and the uninformed channel capacity, with a legend or annotation that labels each curve.

b. What happens to the capacity as the number of multipaths becomes small

In [ ]:
# Scatterer multipath propagation model
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import j0

# parameters
f = 1e9
c0 = 3e8
lam = c0/f
k = 2*np.pi/lam
eta = 377
Gr = 10**(1/10)

## Using the same propagation model from the previous assignment ##

# number of points to simulate
points = 300

# scatter points
x_s = np.random.uniform(-10, 10, points)
y_s = np.random.uniform(-10, 10, points)


# transmitter is an array of Nt elements 
# and the receiver is an array of Nr elements.
# Generate a sequence of a few hundred channel 
# matrices for 4 transmit and 4 receive 
# antennas with optimal element spacing (0.4λ)
Nt = 4      # number transmitting elements
Nr = 4      # number receiving elements
d = 0.4*lam # element spacing

# transmitter location
x_t = 0
y_t = 0

# moving receiver locations
x_r = np.linspace(-10, 10, points)
y_r = np.zeros(points)

# receiver model
Er = np.zeros([points, Nr], dtype=complex)    # received field
ref_coef = -1                               # reflection coefficient

for m in range(0,points):

    # calculate the fields
    for n in range(0,Nr):
        # element positions within the tx array
        x_t_e = x_t + n*d

        # element positions within the moving rx array
        x_r_e = x_r[m] + n*d    
        
        # distance between scatterer and receiver
        delta_r = np.sqrt((x_t_e - x_s[m])**2 + (y_t - y_s[m])**2) \
                - np.sqrt((x_s[m] - x_r_e)**2 + (y_s[m] - y_r[m])**2)
        
        # received field
        Er[m, n] = 1+ref_coef*np.exp(-1j*k*delta_r)

2. Change your model to use circular arrays with 10 elements or so spaced a half wavelength apart so that the arrays can control the radiation patterns in all directions. For a propagation environment model with two or three multipaths, plot the eigenpatterns at the transmit or receive side corresponding to the largest two or three singular values of the channel matrix. Relate these patterns to the physical channel. This can be done by overlaying polar plots of the patterns on an x-y plot of the receiver, transmitter, and scatterer locations.